In [1]:
from utils import *
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
db, engine, meta = db_connect()

In [3]:
tables = db_tables(meta)

In [4]:
leg_map = get_leg_map(engine)

In [9]:
bills_map = get_bills_map(engine)

In [5]:
list(leg_map.values())[0]["congress_map"]

{'110': {'chamber': 'senate',
  'congress': 110,
  'bills_sponsored': 15,
  'total_cosponsors': 28,
  'bills_cosponsored': 61,
  'cosponsors_per_bill': 1.8666666666666667,
  'cosponsored_sponsored_ratio': 4.066666666666666,
  'z_bills_sponsored': -0.833809634522492,
  'z_bills_cosponsored': -1.3986844463050083,
  'z_cosponsors_per_bill': -1.1806980542049446,
  'z_cosponsored_sponsored_ratio': -0.6065545631607324,
  'experience': 0,
  'committee_rank_recips': 0.13392857142857142,
  'committee_min_rank': 14.0,
  'committee_count': 4,
  'max_coeff': -0.05,
  'leadership': 0},
 '111': {'chamber': 'senate',
  'congress': 111,
  'bills_sponsored': 16,
  'total_cosponsors': 31,
  'bills_cosponsored': 88,
  'cosponsors_per_bill': 1.9375,
  'cosponsored_sponsored_ratio': 5.5,
  'z_bills_sponsored': -0.7390674214923396,
  'z_bills_cosponsored': -0.7832512437234525,
  'z_cosponsors_per_bill': -0.5368520518913603,
  'z_cosponsored_sponsored_ratio': -0.11193706915066166,
  'experience': 1,
  'commi

In [6]:
def get_mean_point(pts):
    if len(pts) == 0:
        return None
    return sum(pts) / len(pts)

In [7]:
def pts_variance(pts):
    if len(pts) == 0:
        return None
    mean_point = get_mean_point(pts)
    return sum([np.linalg.norm(p) for p in pts - mean_point]) / len(pts)

In [8]:
def get_nominates(leg_map : dict, legislators : list):
    pts = []
    for leg in legislators:
        if leg["bioguide_id"] in leg_map and "nominate" in leg_map[leg["bioguide_id"]]:
            nominate = leg_map[leg["bioguide_id"]]["nominate"]
            pts.append([nominate["dim1"], nominate["dim2"]])
    return np.array(pts)

In [ ]:
get_bills_count(engine)

In [14]:
# def set_cosponsor_counts(bills : list):
total = get_bills_count(engine)
i = 0
for bill in bills_map.values():
    sponsor = bill["sponsor"]
    if "bioguide_id" not in sponsor:
        continue
    sponsor["party"] = leg_map[sponsor["bioguide_id"]]["party"]
    cosponsor_party_count = {}
    max_cosponsor_experience = -1
    max_cosponsor_committee_rank_recips = 0
    cosponsor_leadership = 0
    for cosponsor in bill["cosponsors"]: 
        if "bioguide_id" not in cosponsor:
            continue
        cosponsor["party"] = leg_map[cosponsor["bioguide_id"]]["party"]
        congress_map = leg_map[cosponsor["bioguide_id"]]["congress_map"]
        max_cosponsor_experience = max(max_cosponsor_experience, congress_map[bill["congress"]]["experience"])
        max_cosponsor_committee_rank_recips = max(max_cosponsor_committee_rank_recips, congress_map[bill["congress"]]["committee_rank_recips"])
        if congress_map[bill["congress"]]["leadership"] == 1:
            cosponsor_leadership = 1
        if cosponsor["party"] not in cosponsor_party_count:
            cosponsor_party_count[cosponsor["party"]] = 0
        cosponsor_party_count[cosponsor["party"]] += 1
    bill["total_cosponsors"] = len(bill["cosponsors"])
    bill["cosponsor_party_count"] = cosponsor_party_count
    bill["nominate_variance"] = pts_variance(get_nominates(leg_map, bill["cosponsors"] + [bill["sponsor"]]))
    bill["max_cosponsor_experience"] = max_cosponsor_experience
    bill["max_cosponsor_committee_rank_recips"] = max_cosponsor_committee_rank_recips
    bill["cosponsor_leadership"] = cosponsor_leadership
    i += 1
    if i % 10000 == 0:
        print("Finished {} of {}".format(i, total))

TypeError: '>' not supported between instances of 'int' and 'NoneType'

In [12]:
bill_infos = []
for bill in bills_map.values():
    sponsor = leg_map[bill["sponsor"]["bioguide_id"]]
    congress_map = sponsor["congress_map"]
    bill_info  = {
        "bill_id" : bill["bill_id"],
        "chamber" : "senate" if bill["bill_type"] == "s" else "house",
        "sponsor_id" : sponsor["id"]["bioguide"],
        "sponsor_party" : sponsor["party"],
        "leadership" : congress_map[bill["congress"]]["leadership"],
        "experience" : congress_map[bill["congress"]]["experience"],
        "committee_count" : congress_map[bill["congress"]]["committee_count"],
        "committee_min_rank" : congress_map[bill["congress"]]["committee_min_rank"] if "committee_min_rank" in congress_map[bill["congress"]] else None,
        "committee_max_coeff" : congress_map[bill["congress"]]["max_coeff"] if "max_coeff" in congress_map[bill["congress"]] else None,
        "committee_rank_recips" : congress_map[bill["congress"]]["committee_rank_recips"],
        "total_cosponsors" : bill["total_cosponsors"],
        "same_party_cosponsors" : bill["cosponsor_party_count"][sponsor["party"]] if sponsor["party"] in bill["cosponsor_party_count"] else 0,
        "nominate_variance" : bill["nominate_variance"],
        "congress" : int(bill["congress"]),
        "enacted" : 0 if bill["enacted_as"] is None else 1,
        "max_cosponsor_experience" : bill["max_cosponsor_experience"],
        "max_cosponsor_committee_rank_recips" : bill["max_cosponsor_committee_rank_recips"],
        "cosponsor_leadership" : bill["cosponsor_leadership"]
    }
    if bill_info["total_cosponsors"] != 0:
        bill_info["same_party_cosponsors_prop"] = bill_info["same_party_cosponsors"] / bill_info["total_cosponsors"]
    bill_infos.append(bill_info)
bill_infos[5], bill_infos[0] = bill_infos[0], bill_infos[5]

In [13]:
save_csv(bill_infos, "data/bill_infos.csv")

In [ ]:
get_one_bill(engine)["congress"]

In [ ]:
sum([bill_info["enacted"] for bill_info in bill_infos]) / len(bill_infos)